# Project 1 

Importing allowed libraries...

In [35]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## I. Load the Data

In [36]:
# Different paths to access data file

Louise_Path = '/Users/louiseplacidet/Desktop/Machine Learning/Project 1/Git_ML_P1/Data/'

In [38]:
def load_data():
    """load data."""
    data = np.genfromtxt(Louise_Path+"train.csv", delimiter=",",dtype='str', skip_header=1, usecols = [])
    ind = data[:,0].astype(np.float)
    x = data[:,2:].astype(np.float)
    y = data[:,1]
    return x, y, ind

In [79]:
x, y, ind = load_data()

In [80]:
x

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

In [81]:
y

array(['s', 'b', 'b', ..., 's', 'b', 'b'], dtype='<U8')

In [82]:
y_binary = y.copy()
y_binary = np.where(y_binary=='b', 1, y_binary)
y_binary = np.where(y_binary=='s', 0, y_binary)
y_binary

array(['0', '1', '1', ..., '0', '1', '1'], dtype='<U8')

In [83]:
y

array(['s', 'b', 'b', ..., 's', 'b', 'b'], dtype='<U8')

In [42]:
ind

array([100000., 100001., 100002., ..., 349997., 349998., 349999.])

## Build Model

In [43]:
num_samples = len(y)
tx = np.c_[np.ones(num_samples), x]

In [44]:
tx

array([[   1.   ,  138.47 ,   51.655, ...,    1.24 ,   -2.475,  113.497],
       [   1.   ,  160.937,   68.768, ..., -999.   , -999.   ,   46.226],
       [   1.   , -999.   ,  162.172, ..., -999.   , -999.   ,   44.251],
       ...,
       [   1.   ,  105.457,   60.526, ..., -999.   , -999.   ,   41.992],
       [   1.   ,   94.951,   19.362, ..., -999.   , -999.   ,    0.   ],
       [   1.   , -999.   ,   72.756, ..., -999.   , -999.   ,    0.   ]])

# II. Implementation Functions

In [46]:
def compute_loss_mse(y, tx, w):
    """Calculate the loss using mse."""
    e = y - tx.dot(w)
    return 1/2 * np.mean(e**2)

### Least Squares GD

In [63]:
def compute_gradient(y, tx, w):
    """Compute a gradient."""
    e = y - tx.dot(w)
    gradient = - (1/len(e)) * tx.T.dot(e)
    
    return gradient, e

In [64]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    losses = []
    w = initial_w
    for n_iter in range(max_iters):

        gradient, e = compute_gradient(y,tx,w)
        loss = compute_loss(y, tx, w)
        
        w = w - gamma * gradient

    return w, loss

### Least Squares SGD

In [65]:
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    e = y - tx.dot(w)
    gradient = - (1/len(e)) * tx.T.dot(e)
    return gradient, e

In [66]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


In [68]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """Stochastic gradient descent algorithm."""
    batch_size = 5 ##TBD
    w = initial_w
    
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y,tx,batch_size = batch_size, num_batches = 1):
            gradient , e = compute_stoch_gradient(y_batch,tx_batch, w)
            w = w - gamma*gradient
            loss = compute_loss(y,tx,w)
            print("loss")
            print(loss)
            print("w")
            print(w)
    
    return w, loss

### Least Squares

In [53]:
def least_squares(y, tx):
    
    w = np.linalg.solve(tx.T@tx,tx.T@y)
    loss = compute_loss_mse(y, tx, w)
    
    return w, loss

### Ridge Regression

In [54]:
def ridge_regression(y, tx, lambda_):

    lambdaI = 2* tx.shape[0] * lambda_ * np.eye(tx.shape[1])
    w = np.linalg.solve(tx.T.dot(tx)+lambdaI,tx.T.dot(y))
    loss = compute_loss_mse(y, tx, w)
    
    return w, loss

### Logistic Regression

##### Sigmoid Function

In [ ]:
def sigmoid(t):
    """apply the sigmoid function on t."""
    return 1 / (1+np.exp(-t))

##### Calculating Loss for logistic regression

In [ ]:
def calculate_loss_logistic(y, tx, w):
    """compute the loss: negative log likelihood."""
    insidesum = np.log(1 + np.exp(tx@w)) - y * (tx@w)
    loss = len(y) * np.mean(insidesum)
    
    return loss

##### Gradient Descent with Logistic Regression

In [ ]:
def calculate_gradient_logistic(y, tx, w):
    """compute the gradient of loss."""
    grad = tx.T @ (sigmoid(tx@w) - y)
    
    return grad

In [ ]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    w = w - gamma * grad
    
    return loss, w

In [55]:
def logistic_regression_gradient(y, tx, initial_w, max_iters, gamma):
    threshold = 1e-8
    
    w = initial_w
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)

        # converge criterion
        #losses.append(loss)
        #if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
        #    break
    return w, loss

##### Newton Method for Logisitic regression

In [ ]:
def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function with respect to parameters w."""

    S = np.eye(len(y))
    S_values = ( sigmoid(tx@w) * (1 - sigmoid(tx@w)))
    
    for i in range(len(y)):
        S[i][i] = S_values[i]
        
    hessian = tx.T @ S @ tx
    return hessian

In [ ]:
def logistic_regression_step(y, tx, w):
    """return the loss, gradient, and Hessian."""

    loss = calculate_loss(y, tx, w)
    gradient = calculate_gradient(y, tx, w)
    hessian = calculate_hessian(y, tx, w)
    
    return loss, gradient, hessian

In [ ]:
def learning_by_newton_method(y, tx, w, gamma = 1):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, gradient, hessian = logistic_regression_step(y, tx, w)
    
    w = w - gamma * np.linalg.inv(hessian) @ gradient
    
    return loss, w

# !!!!!!! WHY DO WE NOT CHOOSE A GAMMA FOR NEWTON METHOD????

In [ ]:
def logistic_regression_newton(y, tx, initial_w, max_iters, gamma):
    threshold = 1e-8
    w = initial_w
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma)

        # converge criterion
        #losses.append(loss)
        #if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
        #    break
    return w, loss

##### Regularized Gradient Descent for Logistic Regression

In [ ]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and Hessian."""
    loss = calculate_loss(y, tx, w) + (lambda_ / 2) * np.sum(w**2)
    gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w
    hessian = calculate_hessian(y, tx, w) + 2 * lambda_
    
    return loss, gradient, hessian

In [ ]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, gradient, _ = penalized_logistic_regression(y, tx, w, lambda_)
    w = w - gamma * gradient
    
    return loss, w

In [57]:
def reg_logistic_regression_gradient(y, tx, lambda_, intial_w, max_iters, gamma):
    threshold = 1e-8
    w = initial_w

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)# !!!!!!! WHY DO WE NOT CHOOSE A GAMMA FOR NEWTON METHOD????    
    
    return w, loss

##### Regularized Newton Method for Logistic Regression

In [ ]:
def penalized_logistic_regression_step(y, tx, w):
    """return the loss, gradient, and Hessian."""

    loss = calculate_loss(y, tx, w) + (lambda_ / 2) * np.sum(w**2)
    gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w
    hessian = calculate_hessian(y, tx, w) + 2 * lambda_
    
    return loss, gradient, hessian

In [ ]:
def reg_logistic_regression_newton(y, tx, initial_w, max_iters, gamma):
    threshold = 1e-8
    w = initial_w
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = penalized_learning_by_newton_method(y, tx, w, gamma)
        
    return w, loss

# III. Implementing the Functions

In [69]:
initial_w = np.array([0, 0])
max_iters = 10
gamma = 0.000001

### Trying Stochastic Gradient Descent

In [ ]:
SGD_loss, SGD_w = least_squares_SGD(y_binary, tx, initial_w, max_iters, gamma)

In [ ]:
SGD_loss

In [ ]:
SGD_w

### Trying Least Squares

In [ ]:
LS_loss, LS_w = least_squares(y_binary, tx)

In [ ]:
LS_loss

In [ ]:
LS_w

### Trying Ridge Regression

In [ ]:
ridge_loss, ridge_w = ridge_regression(y_binary, tx, lambda_)

In [ ]:
ridge_loss

In [ ]:
ridge_w

### Trying Logistic Regression

In [ ]:
logist_loss, logist_w = logistic_regression(y_binary, tx, initial_w, max_iters, gamma)

In [ ]:
logist_loss

In [ ]:
logist_w

### Trying Regularized Logistic Regression

In [ ]:
reg_logist_loss, reg_logist_w = reg_logistic_regression(y_binary, tx, lambda_, intial_w, max_iters, gamma)

In [ ]:
reg_logist_loss

In [ ]:
reg_logist_w

# IV. Preprocessing the Data

# V. Creating Submission CVS

In [ ]:
def calculating_prediction(x_test, weights):
    ##to be completed
    return prediction

In [111]:
def create_submission(identifier,prediction,filename):
    submit_data = np.stack([identifier,prediction],axis=1)
    np.savetxt(filename,submit_data,delimiter=',', fmt='%1.3f',header="Id,Prediction") #fmt 1.3 means I want 3 numbers after decimal point
    

In [112]:
A = np.array([1,2,3])
B = np.array([4,5,6])
C = np.stack([A,B],axis=1)
C

array([[1, 4],
       [2, 5],
       [3, 6]])

In [113]:
filename = "testingsave1.csv"
create_submission(A,B,filename)